In [1]:
#import 부분
import time
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from PIL import ImageOps
import torch
import torch.nn as nn
import pandas as pd
import os
import json
import random
import pickle
from functools import partial
from torchvision.models import resnet50
from torch.utils.data import WeightedRandomSampler
#정확도 및 mae 임포트
from torchmetrics import Accuracy, MeanAbsoluteError
import numpy as np
#import matplotlib.pyplot as plt
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
from sklearn.metrics import mean_absolute_error, accuracy_score
import pandas as pd

c:\Users\yango\anaconda3\envs\eunseo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# YOLO 모델 다운로드 및 로드
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")
yolo_model = YOLO(model_path)


In [4]:
class TestDataset(Dataset):
    def __init__(self, image_dir, label_dir, categories, transform=None, yolo_model=None):
        self.datalist = []
        self.transform = transform
        self.age_min = 10
        self.age_max = 60
        self.yolo_model = yolo_model
        
        for category in categories:
            json_path = os.path.join(label_dir, f'test_{category}.json') 
            img_folder = os.path.join(image_dir, category)
            
            with open(json_path, 'r', encoding='euc-kr') as f:
                label_data = json.load(f)
            
            for row in label_data:
                filename = row['filename']
                base_filename = filename.replace('.jpg', '')
                matched_files = [f for f in os.listdir(img_folder) if f.startswith(base_filename) and f.endswith('.jpg')]
                
                for matched_file in matched_files:
                    img_path = os.path.join(img_folder, matched_file)
                    if not os.path.isfile(img_path):
                        continue
                    
                    age = row.get('age')
                    if age is not None and age >= 60:
                        continue
                    age_norm = (age - self.age_min) / (self.age_max - self.age_min) if age is not None else 0.0
                    
                    data = {
                        'img_path': img_path,
                        'category': category,
                        'age': age_norm,
                        'raw_age': age,
                        'gender': row.get('gender'),
                    }
                    self.datalist.append(data)
    
    def __len__(self):
        return len(self.datalist)
    
    def __getitem__(self, idx):
        data_item = self.datalist[idx]
        image = Image.open(data_item['img_path'])
        image = ImageOps.exif_transpose(image).convert('RGB')

        # YOLO 얼굴 탐지
        results = self.yolo_model(image)
        detections = results[0]  # 첫 번째 이미지 결과

        # confidence 0.7 이상인 얼굴만 필터링
        face_boxes = []
        face_confs = []
        for box, conf in zip(detections.boxes.xyxy, detections.boxes.conf):
            if conf >= 0.7:
                face_boxes.append(box)
                face_confs.append(conf)

        if len(face_boxes) == 0:
            # 얼굴 없으면 원본 이미지 리사이즈
            face_img = image.resize((224, 224))
        else:
            # 가장 높은 confidence 얼굴 1개만 선택
            max_idx = face_confs.index(max(face_confs))
            x1, y1, x2, y2 = map(int, face_boxes[max_idx])
            face_img = image.crop((x1, y1, x2, y2)).resize((224, 224))

        if self.transform:
            face_img = self.transform(face_img)

        age = torch.tensor(data_item['age'], dtype=torch.float32)
        gender = torch.tensor(1 if data_item['gender'] == '남' else 0, dtype=torch.long)

        return face_img, age, gender

In [ ]:
# 모델 불러오기 (resnet50 기반)
model = resnet50()
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.4),

    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.3),

    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(64, 3)  # age(1), gender(2)
)
base_dir = os.path.dirname(os.path.abspath(__file__))
weight_path = os.path.join(base_dir, 'pth_pkl', 'model_aug_weights_v2_2_1_6.pth')

model.load_state_dict(torch.load(weight_path, map_location=device))
model.to(device)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
# 전처리 transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
categories = ['anger','happy','panic','sadness']
base_dir = os.path.dirname(os.path.abspath(__file__))
test_image_dir = os.path.join(base_dir, 'test', 'image')
test_label_dir = os.path.join(base_dir, 'test', 'label')

In [8]:
test_dataset = TestDataset(test_image_dir, test_label_dir, categories, transform=transform, yolo_model=yolo_model)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [9]:
# 누적할 리스트 생성
all_true_ages = []
all_pred_ages = []
all_true_genders = []
all_pred_genders = []

# 5. 추론 + 평가
with torch.no_grad():
    for images, ages, genders in test_loader:
        images = images.to(device)
        ages = ages.to(device)
        genders = genders.to(device)

        outputs = model(images)
        pred_age = outputs[:,0]
        pred_gender_logits = outputs[:,1:3]

        # 나이 역정규화
        pred_age_orig = pred_age * (test_dataset.age_max - test_dataset.age_min) + test_dataset.age_min
        true_age_orig = ages * (test_dataset.age_max - test_dataset.age_min) + test_dataset.age_min

        pred_gender = torch.argmax(pred_gender_logits, dim=1)

        # 리스트에 누적
        all_pred_ages.extend(pred_age_orig.cpu().numpy())
        all_true_ages.extend(true_age_orig.cpu().numpy())
        all_pred_genders.extend(pred_gender.cpu().numpy())
        all_true_genders.extend(genders.cpu().numpy())

        # 샘플 출력
        for i in range(len(images)):
            pred_gender_str = '남' if pred_gender[i].item() == 1 else '여'
            true_gender_str = '남' if genders[i].item() == 1 else '여'
            print(f'predicted age : {int(round(pred_age_orig[i].item()))}세, actual age: {int(round(true_age_orig[i].item()))}세')
            print(f'predicted gender: {pred_gender_str}, actual gender: {true_gender_str}')

# 전체 결과 계산
test_mae = mean_absolute_error(all_true_ages, all_pred_ages)
test_acc = accuracy_score(all_true_genders, all_pred_genders)

print('\n 테스트 평가 결과')
print(f'나이 MAE: {test_mae:.4f}')
print(f'성별 정확도: {test_acc:.4f}')


0: 480x640 1 FACE, 77.7ms
Speed: 4.2ms preprocess, 77.7ms inference, 265.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 FACE, 6.7ms
Speed: 1.7ms preprocess, 6.7ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 FACE, 6.4ms
Speed: 2.2ms preprocess, 6.4ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 384x640 1 FACE, 78.9ms
Speed: 1.5ms preprocess, 78.9ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 FACE, 6.9ms
Speed: 2.0ms preprocess, 6.9ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 FACE, 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 FACE, 6.2ms
Speed: 1.4ms preprocess, 6.2ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 FACE, 5.5ms
Speed: 1.7ms preprocess, 5.5ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 FACE

In [ ]:
# 성별 숫자 → 문자 변환 함수
def gender_to_str(gender_int):
    return '남' if gender_int == 1 else '여'

# DataFrame 만들기
import numpy as np
df = pd.DataFrame({
    '예측 나이': np.round(all_pred_ages).astype(int),
    '실제 나이': np.round(all_true_ages).astype(int),
    '예측 성별': [gender_to_str(g) for g in all_pred_genders],
    '실제 성별': [gender_to_str(g) for g in all_true_genders]
})

# CSV 저장 (utf-8-sig로 한글 깨짐 방지)
base_dir = os.path.dirname(os.path.abspath(__file__))
save_path = os.path.join(base_dir, 'test_result', 'test_v2_2_1_6_int.csv')

df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"CSV 파일로 결과 저장 완료: {save_path}")

CSV 파일로 결과 저장 완료: test_v2_2_1_6_int.csv
